In [46]:
import pandas as pd

df = pd.read_csv('sbb-data_latlong.csv', delimiter=',')

#Conversion en format date
df['Departure forecast'] = pd.to_datetime(df['Departure forecast'], errors='coerce')
df['Departure time'] = pd.to_datetime(df['Departure time'], errors='coerce')
df['Arrival forecast'] = pd.to_datetime(df['Arrival forecast'], errors='coerce')
df['Arrival time'] = pd.to_datetime(df['Arrival time'], errors='coerce')

#Calcul des temps de retard, si calcul pas pertinent retourne Nat
df['Arrival time delay'] = df.apply(
    lambda row: row['Arrival forecast'] - row['Arrival time'] 
                if pd.notna(row['Arrival forecast']) and pd.notna(row['Arrival time']) and row['Arrival forecast'] > row['Arrival time']
                else pd.NaT,
    axis=1
)
df['Departure time delay'] = df.apply(
    lambda row: row['Departure forecast'] - row['Departure time'] 
                if pd.notna(row['Departure forecast']) and pd.notna(row['Departure time']) and row['Departure forecast'] > row['Departure time']
                else pd.NaT,
    axis=1
)

In [47]:
pd.set_option('display.max_columns', None)
df.head(10)

,Day of operation,Journey identifier,Operator ID,Operator abbreviation,Operator name,Product ID,Linie,Line Text,Rotation ID,Transport ID,Additional run TF,Cancelled TF,OPUIC,Stop name,Arrival time,Arrival forecast,Arrival forecast status,Departure time,Departure forecast,Departure forecast status,Non-stopping pass TF,Arrival delay,Departure delay,Geopos,lod,latitude,longitude,Arrival time delay,Departure time delay
0,2024-10-13,ch:1:sjyid:100001:19189-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,19189,S11,NaN,S,False,True,8506016,Oberwinterthur,2024-10-13 23:29:00,NaT,UNBEKANNT,2024-10-13 23:30:00,NaT,UNBEKANNT,False,False,False,"47.5079223325029, 8.760389196167752",http://lod.opentransportdata.swiss/didok/didok85,47.507922,8.760389,NaT,NaT
1,2024-10-13,ch:1:sjyid:100001:19190-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,19190,S11,NaN,S,False,True,8503003,Zürich Stadelhofen,2024-10-13 23:25:00,NaT,UNBEKANNT,2024-10-13 23:25:00,NaT,UNBEKANNT,False,False,False,"47.36660976671019, 8.548489088449456",http://lod.opentransportdata.swiss/didok/didok85,47.366610,8.548489,NaT,NaT
2,2024-10-13,ch:1:sjyid:100001:19191-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,19191,S11,NaN,S,False,False,8502105,Othmarsingen,2024-10-13 22:58:00,2024-10-13 22:58:52,REAL,2024-10-13 22:58:00,2024-10-13 22:59:33,REAL,False,False,False,"47.40743350791071, 8.214805494441988",http://lod.opentransportdata.swiss/didok/didok85,47.407434,8.214805,0 days 00:00:52,0 days 00:01:33
3,2024-10-13,ch:1:sjyid:100001:19191-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,19191,S11,NaN,S,False,False,8503509,Schlieren,2024-10-13 23:19:00,2024-10-13 23:20:00,REAL,2024-10-13 23:19:00,2024-10-13 23:21:11,REAL,False,False,False,"47.39917636823757, 8.44724506333892",http://lod.opentransportdata.swiss/didok/didok85,47.399176,8.447245,0 days 00:01:00,0 days 00:02:11
4,2024-10-13,ch:1:sjyid:100001:19192-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,19192,S11,NaN,S,False,False,8516219,Mellingen Heitersberg,2024-10-14 00:24:00,2024-10-14 00:24:44,REAL,2024-10-14 00:24:00,2024-10-14 00:25:32,REAL,False,False,False,"47.42807508777434, 8.276363251209915",http://lod.opentransportdata.swiss/didok/didok85,47.428075,8.276363,0 days 00:00:44,0 days 00:01:32
5,2024-10-13,ch:1:sjyid:100001:19193-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,19193,S11,NaN,S,False,False,8503001,Zürich Altstetten,2024-10-13 23:53:00,2024-10-13 23:53:37,REAL,2024-10-13 23:54:00,2024-10-13 23:54:29,REAL,False,False,False,"47.3914830730815, 8.488962649961255",http://lod.opentransportdata.swiss/didok/didok85,47.391483,8.488963,0 days 00:00:37,0 days 00:00:29
6,2024-10-13,ch:1:sjyid:100001:19195-002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,19195,S11,NaN,S,False,False,8503508,Dietikon,2024-10-14 00:14:00,2024-10-14 00:16:39,REAL,2024-10-14 00:15:00,2024-10-14 00:16:50,REAL,False,False,False,"47.40578380656623, 8.404988187894768",http://lod.opentransportdata.swiss/didok/didok85,47.405784,8.404988,0 days 00:02:39,0 days 00:01:50
7,2024-10-13,ch:1:sjyid:100001:19214-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,19214,S12,NaN,S,False,False,8503505,Wettingen,2024-10-13 05:09:00,2024-10-13 05:09:01,REAL,2024-10-13 05:10:00,2024-10-13 05:10:10,REAL,False,False,False,"47.45963817592964, 8.316006171449603",http://lod.opentransportdata.swiss/didok/didok85,47.459638,8.316006,0 days 00:00:01,0 days 00:00:10
8,2024-10-13,ch:1:sjyid:100001:19217-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,19217,S12,NaN,S,False,False,8503509,Schlieren,2024-10-13 05:04:00,2024-10-13 05:04:24,REAL,2024-10-13 05:04:00,2024-10-13 05:05:15,REAL,False,False,False,"47.39917636823757, 8.44724506333892",http://lod.opentransportdata.swiss/didok/didok85,47.399176,8.447245,0 days 00:00:24,0 days 00:01:15
9,2024-10-13,ch:1:sjyid:100001:19218-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,19218,S12,NaN,S,False,False,8503512,Glanzenberg,2024-10-13 05:56:00,2024-10-13 05:55:58,REAL,2024-10-13 05:56:00,2024-10-13 05:56:49,REA

In [48]:
departure_delay_counts = df[df['Departure delay'] == True].groupby('Stop name').size().reindex(stop_names, fill_value=0)
arrival_delay_counts = df[df['Arrival delay'] == True].groupby('Stop name').size().reindex(stop_names, fill_value=0)
train_counts = df.groupby('Stop name').size().reindex(stop_names)
max_departure_time_delay = df.groupby('Stop name')['Departure time delay'].max().reindex(stop_names)
max_arrival_time_delay = df.groupby('Stop name')['Arrival time delay'].max().reindex(stop_names)


print("departure_delay_counts : \n", departure_delay_counts, "\n")
print("arrival_delay_counts : \n", arrival_delay_counts, "\n")
print("train_count : \n", train_count, "\n")
print("max_departure_time_delay : \n", max_departure_time_delay, "\n")
print("max_arrival_time_delay : \n", max_arrival_time_delay, "\n")

departure_delay_counts : 
 Stop name
Oberwinterthur            0
Zürich Stadelhofen       19
Othmarsingen              9
Schlieren                 6
Mellingen Heitersberg     8
                         ..
Maienfeld                 1
Wynigen                   1
Burgdorf                  0
Ambrì-Piotta              0
Düdingen                  0
Length: 586, dtype: int64 

arrival_delay_counts : 
 Stop name
Oberwinterthur            0
Zürich Stadelhofen       15
Othmarsingen              7
Schlieren                 4
Mellingen Heitersberg     8
                         ..
Maienfeld                 0
Wynigen                   1
Burgdorf                  0
Ambrì-Piotta              0
Düdingen                  0
Length: 586, dtype: int64 

train_count : 
 Stop name
Oberwinterthur            72
Zürich Stadelhofen       719
Othmarsingen             157
Schlieren                165
Mellingen Heitersberg     78
                        ... 
Maienfeld                  3
Wynigen                    

In [58]:
latitude = df.groupby('Stop name')['latitude'].max().reindex(stop_names)
longitude = df.groupby('Stop name')['longitude'].max().reindex(stop_names)

print(latitude)
print(longitude)

Stop name
Oberwinterthur           47.507922
Zürich Stadelhofen       47.366610
Othmarsingen             47.407434
Schlieren                47.399176
Mellingen Heitersberg    47.428075
                           ...    
Maienfeld                47.003841
Wynigen                  47.105175
Burgdorf                 47.060701
Ambrì-Piotta             46.510764
Düdingen                 46.851052
Name: latitude, Length: 586, dtype: float64
Stop name
Oberwinterthur           8.760389
Zürich Stadelhofen       8.548489
Othmarsingen             8.214805
Schlieren                8.447245
Mellingen Heitersberg    8.276363
                           ...   
Maienfeld                9.529216
Wynigen                  7.663507
Burgdorf                 7.621694
Ambrì-Piotta             8.689928
Düdingen                 7.189753
Name: longitude, Length: 586, dtype: float64


In [63]:
#merge dataframes
merged_df = pd.concat([departure_delay_counts, arrival_delay_counts, train_counts, max_departure_time_delay, max_arrival_time_delay, 
                       latitude, longitude], axis=1)
merged_df.columns = ['Departure delay count', 'Arrival delay count', 'Train count', 'Max departure time delay', 'Max arrival time delay',
                    'latitude','longitude']


print(merged_df)

                       Departure delay count  Arrival delay count  \
Stop name                                                           
Oberwinterthur                             0                    0   
Zürich Stadelhofen                        19                   15   
Othmarsingen                               9                    7   
Schlieren                                  6                    4   
Mellingen Heitersberg                      8                    8   
...                                      ...                  ...   
Maienfeld                                  1                    0   
Wynigen                                    1                    1   
Burgdorf                                   0                    0   
Ambrì-Piotta                               0                    0   
Düdingen                                   0                    0   

                       Train count Max departure time delay  \
Stop name                              

In [64]:
merged_df.to_csv('train_stations_data.csv', index=True)